# Getting Started With Async Features in Python
> https://realpython.com/python-async-features/

## Synchronous Programming

In [4]:
import queue

In [2]:
def task(name, work_queue):
    if work_queue.empty():
        print(f"Task {name} nothing to do")
    else:
        while not work_queue.empty():
            count = work_queue.get()
            total = 0
            print(f"Task {name} running")
            for x in range(count):
                total += 1
            print(f"Task {name} total: {total}")

In [5]:
work_queue = queue.Queue()

In [6]:
for work in [15, 10, 5, 2]:
    work_queue.put(work)

In [7]:
tasks = [(task, "One", work_queue), (task, "Two", work_queue)]

In [8]:
for t, n, q in tasks:
    t(n,q)

Task One running
Task One total: 15
Task One running
Task One total: 10
Task One running
Task One total: 5
Task One running
Task One total: 2
Task Two nothing to do


## Simple Cooperative Concurrency

In [9]:
import queue

In [74]:
def task(name, queue):
    while not queue.empty():
        count = queue.get()
        total = 0
        print(f"Task {name} count: {count}")
        print(f"Task {name} running")
        for x in range(count):
            print(f"Task {name} x: {x}")
            total += 1
            yield
        print(f"Task {name} total: {total}")

In [75]:
work_queue = queue.Queue()

In [76]:
for work in [15, 10, 5, 2]:
    work_queue.put(work)

In [77]:
tasks = [task("one", work_queue), task("Two", work_queue)]

In [78]:
done = False

In [79]:
while not done:
    print('--while--')
    for t in tasks:
        print('---for--')
        try:
            next(t)
        except StopIteration:
            print('except here')
            tasks.remove(t)
        if len(tasks) == 0:
            done = True

--while--
---for--
Task one count: 15
Task one running
Task one x: 0
---for--
Task Two count: 10
Task Two running
Task Two x: 0
--while--
---for--
Task one x: 1
---for--
Task Two x: 1
--while--
---for--
Task one x: 2
---for--
Task Two x: 2
--while--
---for--
Task one x: 3
---for--
Task Two x: 3
--while--
---for--
Task one x: 4
---for--
Task Two x: 4
--while--
---for--
Task one x: 5
---for--
Task Two x: 5
--while--
---for--
Task one x: 6
---for--
Task Two x: 6
--while--
---for--
Task one x: 7
---for--
Task Two x: 7
--while--
---for--
Task one x: 8
---for--
Task Two x: 8
--while--
---for--
Task one x: 9
---for--
Task Two x: 9
--while--
---for--
Task one x: 10
---for--
Task Two total: 10
Task Two count: 5
Task Two running
Task Two x: 0
--while--
---for--
Task one x: 11
---for--
Task Two x: 1
--while--
---for--
Task one x: 12
---for--
Task Two x: 2
--while--
---for--
Task one x: 13
---for--
Task Two x: 3
--while--
---for--
Task one x: 14
---for--
Task Two x: 4
--while--
---for--
Task one t

## Cooperative Concurrency With Blocking Calls

In [80]:
import time
import queue
from codetiming import Timer

In [86]:
def task(name, queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    while not queue.empty():
        delay = queue.get()
        print(f"Task {name} running")
        timer.start()
        time.sleep(delay)
        timer.stop()
        yield

In [87]:
work_queue = queue.Queue()

In [88]:
for work in [15, 10, 5, 2]:
    work_queue.put(work)

In [89]:
tasks = [task("One", work_queue), task("Two", work_queue)]

In [90]:
done = False
with Timer(text="\nTotal elasped time: {:.1f}"):
    while not done:
        for t in tasks:
            try:
                next(t)
            except StopIteration:
                tasks.remove(t)
            if len(tasks) == 0:
                done = True

Task One running
Task One elapsed time: 15.0
Task Two running
Task Two elapsed time: 10.0
Task One running
Task One elapsed time: 5.0
Task Two running
Task Two elapsed time: 2.0

Total elasped time: 32.0


## Cooperative Concurrency With Non-Blocking Calls

The event loop is at the heart of the Python async system. It runs all the code, including `main()`. When task code is executing, the CPU is busy doing work. When the `await` keyword is reached, a context switch occurs, and control passes back to the event loop. The event loop looks at all the tasks wating for an event and passes control to a task with an event that's ready.

In [92]:
import asyncio
from codetiming import Timer

In [93]:
async def task(name, work_queue):
    timer = Timer(text=f"Task {name} elasped time: {{:.1f}}")
    while not work_queue.empty():
        delay = await work_queue.get()
        print(f"Task {name} running")
        timer.start()
        await asyncio.sleep(delay)
        timer.stop()

In [94]:
async def main():
    work_queue = asyncio.Queue()

    for work in [15, 10, 5, 2]:
        await work_queue.put(work)
    
    with Timer(text="\nTotal elapsed time: {:.1f}"):
        await asyncio.gather(
            asyncio.create_task(task("One", work_queue)),
            asyncio.create_task(task("Two", work_queue)),
        )

In [96]:
await main()

Task One running
Task Two running
Task Two elasped time: 10.0
Task Two running
Task One elasped time: 15.0
Task One running
Task Two elasped time: 5.0
Task One elasped time: 2.0

Total elapsed time: 17.0


## Synchronous (Blocking) HTTP Calls

In [97]:
import queue
import requests
from codetiming import Timer

In [98]:
def task(name, work_queue):
    timer = Timer(text=f"Task {name} elasped time: {{:.1f}}")
    with requests.Session() as session:
        while not work_queue.empty():
            url = work_queue.get()
            print(f"Task {name} getting URL: {url}")
            timer.start()
            session.get(url)
            timer.stop()
            yield

In [102]:
def main():
    work_queue = queue.Queue()

    for url in [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com",
        "http://twitter.com",
    ]:
        work_queue.put(url)
    
    tasks = [task("One", work_queue), task("Two", work_queue)]

    done = False
    with Timer(text="\nTotal elasped time: {:.1f}"):
        while not done:
            for t in tasks:
                try:
                    next(t)
                except StopIteration:
                    tasks.remove(t)
                if len(tasks) == 0:
                    done = True

In [103]:
main()

Task One getting URL: http://google.com
Task One elasped time: 0.6
Task Two getting URL: http://yahoo.com
Task Two elasped time: 2.0
Task One getting URL: http://linkedin.com
Task One elasped time: 0.8
Task Two getting URL: http://apple.com
Task Two elasped time: 0.3
Task One getting URL: http://microsoft.com
Task One elasped time: 0.4
Task Two getting URL: http://facebook.com
Task Two elasped time: 2.2
Task One getting URL: http://twitter.com
Task One elasped time: 0.7

Total elasped time: 7.1


## Asynchronous (Non-Blocking) HTTP Calls

In [105]:
import asyncio
import aiohttp
from codetiming import Timer

In [110]:
async def task(name, work_queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    async with aiohttp.ClientSession() as session:
        while not work_queue.empty():
            url = await work_queue.get()
            print(f"Task {name} getting URL: {url}")
            timer.start()
            async with session.get(url) as response:
                await response.text()
            timer.stop()

In [111]:
async def main():
    work_queue = asyncio.Queue()

    for url in [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com",
        "http://twitter.com",
    ]:
        await work_queue.put(url)
    
    with Timer(text="\nTotal elasped time: {:.1f}"):
        await asyncio.gather(
            asyncio.create_task(task("One", work_queue)),
            asyncio.create_task(task("Two", work_queue))
        )

In [112]:
await main()

Task One getting URL: http://google.com
Task Two getting URL: http://yahoo.com
Task One elapsed time: 0.4
Task One getting URL: http://linkedin.com
Task One elapsed time: 2.5
Task One getting URL: http://apple.com
Task Two elapsed time: 2.9
Task Two getting URL: http://microsoft.com
Task One elapsed time: 0.4
Task One getting URL: http://facebook.com
Task Two elapsed time: 1.1
Task Two getting URL: http://twitter.com
Task Two elapsed time: 1.4
Task One elapsed time: 3.3

Total elasped time: 6.5
